In [1]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

import pandas as pd
import numpy as np

from pathlib import Path
from datetime import datetime
import multiprocessing
import logging

cores = multiprocessing.cpu_count()
print(f'cores: {cores}')

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

cores: 4


In [2]:
path_data = Path('../../data/corpora/clean_tutby_126784.csv')
# path_data = Path('../../data/corpora/clean_tutby_126784_header.csv')

path_d2v = Path('../../data/model/d2v/model_d2v_clean_tutby_126784.model')
# path_d2v = Path('../../data/model/d2v/model_d2v_clean_tutby_126784_header.model')

path_embedding = Path('../../data/emb/emb_clean_tutby_126784_d2v.npy')
# path_embedding = Path('../../data/emb/emb_clean_tutby_126784_header_d2v.npy')

column_name = 'document'
# column_name = 'header'

In [3]:
data = pd.read_csv(path_data)

documents = data[column_name]
documents = documents.fillna('')

print(data.shape)
data.head(5)

(126784, 6)


,url,label,header,date,document,tags
0,https://news.tut.by/550306.html,Футбол,"Тренер ""Шахтера"": Оправдываться не хочу. Все в...",2017-07-06T21:35:00+03:00,главный тренер солигорский шахтер олег кубарев...,['футбол']
1,https://news.tut.by/550307.html,Общество,"""Зацветет"" ли каменная роза на ул. Комсомольск...",2017-07-07T09:25:00+03:00,план восстановление рисунок пока художник илья...,"['архитектура', 'живопись', 'ЖКХ']"
2,https://news.tut.by/550308.html,Общество,Фотофакт. Скамейка в виде пожарной машины появ...,2017-07-07T09:27:00+03:00,областной управление мчс день пожарный служба ...,['министерства']
3,https://news.tut.by/550309.html,Футбол,Станислав Драгун дебютировал за БАТЭ в матче с...,2017-07-06T22:11:00+03:00,чемпион беларусь бате воспользоваться пауза че...,"['футбол', 'БАТЭ']"
4,https://news.tut.by/550310.html,В мире,Генпрокурор Украины пообещал открыть уголовное...,2017-07-06T22:28:00+03:00,генпрокуратура украина открывать уголовный про...,"['Ситуация в Украине', 'государственные перево..."


In [ ]:
%%time
print(datetime.now())

params_doc2vec = {
    'dm': 0,
    'vector_size': 300,
    'window': 15,
    'min_count': 20,
    'sample': 1e-5,
    'workers': cores,
    'epochs': 40,
    'hs': 0,
    'negative': 5,
    'dbow_words': 1,
}

documents_gensim = [TaggedDocument(document.split(), [index]) for index, document in documents.items()]

model = Doc2Vec(documents=documents_gensim, **params_doc2vec)

model.save(str(path_d2v))
model = Doc2Vec.load(str(path_d2v))

embeddings = model.docvecs.vectors_docs

with open(path_embedding, 'wb') as file:
    np.save(file, embeddings)

with open(path_embedding, 'rb') as file:
    embeddings = np.load(file)
    
print(embeddings.shape)

2020-11-15 00:44:02.203523


2020-11-15 00:44:09,487 : INFO : collecting all words and their counts
2020-11-15 00:44:09,492 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-11-15 00:44:10,506 : INFO : PROGRESS: at example #10000, processed 2466256 words (2432683/s), 99885 word types, 10000 tags
2020-11-15 00:44:11,537 : INFO : PROGRESS: at example #20000, processed 5035139 words (2494384/s), 143097 word types, 20000 tags
2020-11-15 00:44:12,721 : INFO : PROGRESS: at example #30000, processed 7623391 words (2189747/s), 176652 word types, 30000 tags
2020-11-15 00:44:13,500 : INFO : PROGRESS: at example #40000, processed 10215111 words (3331902/s), 205336 word types, 40000 tags
2020-11-15 00:44:14,199 : INFO : PROGRESS: at example #50000, processed 12787803 words (3684520/s), 231745 word types, 50000 tags
2020-11-15 00:44:14,809 : INFO : PROGRESS: at example #60000, processed 15423913 words (4327137/s), 256913 word types, 60000 tags
2020-11-15 00:44:15,415 : INFO : PROGRESS: at exa

2020-11-15 00:45:47,218 : INFO : EPOCH 1 - PROGRESS: at 20.68% examples, 48753 words/s, in_qsize 8, out_qsize 0
2020-11-15 00:45:48,286 : INFO : EPOCH 1 - PROGRESS: at 21.10% examples, 48799 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:45:49,447 : INFO : EPOCH 1 - PROGRESS: at 21.59% examples, 48888 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:45:50,498 : INFO : EPOCH 1 - PROGRESS: at 21.94% examples, 48796 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:45:51,720 : INFO : EPOCH 1 - PROGRESS: at 22.27% examples, 48484 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:45:52,769 : INFO : EPOCH 1 - PROGRESS: at 22.57% examples, 48334 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:45:53,903 : INFO : EPOCH 1 - PROGRESS: at 22.99% examples, 48247 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:45:54,979 : INFO : EPOCH 1 - PROGRESS: at 23.31% examples, 48144 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:45:55,995 : INFO : EPOCH 1 - PROGRESS: at 23.59% examples, 47973 words/s, in_qsize 7, out_

2020-11-15 00:47:07,242 : INFO : EPOCH 1 - PROGRESS: at 49.06% examples, 47180 words/s, in_qsize 8, out_qsize 0
2020-11-15 00:47:08,327 : INFO : EPOCH 1 - PROGRESS: at 49.53% examples, 47257 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:47:09,392 : INFO : EPOCH 1 - PROGRESS: at 50.00% examples, 47322 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:47:10,427 : INFO : EPOCH 1 - PROGRESS: at 50.43% examples, 47393 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:47:11,475 : INFO : EPOCH 1 - PROGRESS: at 50.88% examples, 47448 words/s, in_qsize 8, out_qsize 0
2020-11-15 00:47:12,484 : INFO : EPOCH 1 - PROGRESS: at 51.30% examples, 47491 words/s, in_qsize 6, out_qsize 1
2020-11-15 00:47:13,550 : INFO : EPOCH 1 - PROGRESS: at 51.75% examples, 47565 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:47:14,567 : INFO : EPOCH 1 - PROGRESS: at 52.18% examples, 47627 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:47:15,576 : INFO : EPOCH 1 - PROGRESS: at 52.62% examples, 47673 words/s, in_qsize 7, out_

2020-11-15 00:48:25,626 : INFO : EPOCH 1 - PROGRESS: at 76.61% examples, 46756 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:48:26,758 : INFO : EPOCH 1 - PROGRESS: at 76.76% examples, 46618 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:48:27,783 : INFO : EPOCH 1 - PROGRESS: at 76.93% examples, 46518 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:48:28,981 : INFO : EPOCH 1 - PROGRESS: at 77.15% examples, 46385 words/s, in_qsize 8, out_qsize 0
2020-11-15 00:48:30,051 : INFO : EPOCH 1 - PROGRESS: at 77.38% examples, 46309 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:48:31,080 : INFO : EPOCH 1 - PROGRESS: at 77.65% examples, 46267 words/s, in_qsize 8, out_qsize 0
2020-11-15 00:48:32,194 : INFO : EPOCH 1 - PROGRESS: at 77.90% examples, 46184 words/s, in_qsize 7, out_qsize 0
2020-11-15 00:48:33,289 : INFO : EPOCH 1 - PROGRESS: at 78.25% examples, 46145 words/s, in_qsize 8, out_qsize 0
2020-11-15 00:48:34,408 : INFO : EPOCH 1 - PROGRESS: at 78.59% examples, 46121 words/s, in_qsize 7, out_

...

2020-11-15 01:38:39,758 : INFO : EPOCH 13 - PROGRESS: at 45.84% examples, 54703 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:38:40,804 : INFO : EPOCH 13 - PROGRESS: at 46.31% examples, 54736 words/s, in_qsize 8, out_qsize 0
2020-11-15 01:38:41,845 : INFO : EPOCH 13 - PROGRESS: at 46.71% examples, 54743 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:38:42,916 : INFO : EPOCH 13 - PROGRESS: at 47.21% examples, 54759 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:38:43,948 : INFO : EPOCH 13 - PROGRESS: at 47.66% examples, 54781 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:38:45,038 : INFO : EPOCH 13 - PROGRESS: at 48.13% examples, 54796 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:38:46,047 : INFO : EPOCH 13 - PROGRESS: at 48.56% examples, 54817 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:38:47,157 : INFO : EPOCH 13 - PROGRESS: at 49.02% examples, 54821 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:38:48,332 : INFO : EPOCH 13 - PROGRESS: at 49.50% examples, 54795 words/s, in_qsiz

2020-11-15 01:39:57,322 : INFO : EPOCH 13 - PROGRESS: at 78.01% examples, 54623 words/s, in_qsize 8, out_qsize 0
2020-11-15 01:39:58,347 : INFO : EPOCH 13 - PROGRESS: at 78.50% examples, 54650 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:39:59,428 : INFO : EPOCH 13 - PROGRESS: at 79.02% examples, 54664 words/s, in_qsize 8, out_qsize 0
2020-11-15 01:40:00,511 : INFO : EPOCH 13 - PROGRESS: at 79.47% examples, 54672 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:40:01,582 : INFO : EPOCH 13 - PROGRESS: at 79.93% examples, 54682 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:40:02,672 : INFO : EPOCH 13 - PROGRESS: at 80.34% examples, 54690 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:40:03,700 : INFO : EPOCH 13 - PROGRESS: at 80.76% examples, 54696 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:40:04,791 : INFO : EPOCH 13 - PROGRESS: at 81.19% examples, 54700 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:40:05,793 : INFO : EPOCH 13 - PROGRESS: at 81.57% examples, 54689 words/s, in_qsiz

2020-11-15 01:41:10,704 : INFO : EPOCH 14 - PROGRESS: at 9.05% examples, 56025 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:41:11,727 : INFO : EPOCH 14 - PROGRESS: at 9.44% examples, 55735 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:41:12,841 : INFO : EPOCH 14 - PROGRESS: at 9.92% examples, 55701 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:41:13,844 : INFO : EPOCH 14 - PROGRESS: at 10.37% examples, 55830 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:41:14,860 : INFO : EPOCH 14 - PROGRESS: at 10.77% examples, 55825 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:41:15,904 : INFO : EPOCH 14 - PROGRESS: at 11.21% examples, 55673 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:41:16,923 : INFO : EPOCH 14 - PROGRESS: at 11.68% examples, 55705 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:41:18,043 : INFO : EPOCH 14 - PROGRESS: at 12.17% examples, 55696 words/s, in_qsize 8, out_qsize 0
2020-11-15 01:41:19,186 : INFO : EPOCH 14 - PROGRESS: at 12.65% examples, 55675 words/s, in_qsize 7

2020-11-15 01:42:28,959 : INFO : EPOCH 14 - PROGRESS: at 37.71% examples, 49177 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:42:29,992 : INFO : EPOCH 14 - PROGRESS: at 37.96% examples, 48975 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:42:31,017 : INFO : EPOCH 14 - PROGRESS: at 38.27% examples, 48858 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:42:32,033 : INFO : EPOCH 14 - PROGRESS: at 38.53% examples, 48700 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:42:33,069 : INFO : EPOCH 14 - PROGRESS: at 38.74% examples, 48471 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:42:34,110 : INFO : EPOCH 14 - PROGRESS: at 39.00% examples, 48324 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:42:35,198 : INFO : EPOCH 14 - PROGRESS: at 39.40% examples, 48261 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:42:36,221 : INFO : EPOCH 14 - PROGRESS: at 39.71% examples, 48162 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:42:37,290 : INFO : EPOCH 14 - PROGRESS: at 40.11% examples, 48148 words/s, in_qsiz

2020-11-15 01:43:47,145 : INFO : EPOCH 14 - PROGRESS: at 60.98% examples, 44648 words/s, in_qsize 8, out_qsize 0
2020-11-15 01:43:48,295 : INFO : EPOCH 14 - PROGRESS: at 61.41% examples, 44704 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:43:49,313 : INFO : EPOCH 14 - PROGRESS: at 61.85% examples, 44779 words/s, in_qsize 8, out_qsize 0
2020-11-15 01:43:50,378 : INFO : EPOCH 14 - PROGRESS: at 62.28% examples, 44831 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:43:51,399 : INFO : EPOCH 14 - PROGRESS: at 62.70% examples, 44886 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:43:52,471 : INFO : EPOCH 14 - PROGRESS: at 63.17% examples, 44932 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:43:53,495 : INFO : EPOCH 14 - PROGRESS: at 63.63% examples, 44991 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:43:54,528 : INFO : EPOCH 14 - PROGRESS: at 64.04% examples, 45011 words/s, in_qsize 7, out_qsize 0
2020-11-15 01:43:55,655 : INFO : EPOCH 14 - PROGRESS: at 64.52% examples, 45063 words/s, in_qsiz